# Machine Reading: Advanced Topics in Word Vectors
## Part IV. Role of Bias in Word Embeddings (50 mins)

This is a 4-part series of Jupyter notebooks on the topic of word embeddings originally created for a workshop during the Digital Humanities 2018 Conference in Mexico City. Each part is comprised of a mix of theoretical explanations and fill-in-the-blanks activities of increasing difficulty.

Instructors:
- Eun Seo Jo, <a href="mailto:eunseo@stanford.edu">*eunseo@stanford.edu*</a>, Stanford University
- Javier de la Rosa, <a href="mailto:versae@stanford.edu">*versae@stanford.edu*</a>, Stanford University
- Scott Bailey, <a href="mailto:scottbailey@stanford.edu">*scottbailey@stanford.edu*</a>, Stanford University

In this unit, we will explore an application and caveat of using word embeddings -- cultural bias. Presenting methods and results from recent articles, we will show how word embeddings can carry the historical bias of the training corpora and lead an activity that shows these human-biases on vectors. We'll also address how such bias can be mitigated.

- 0:00 - 0:10 Algorithmic bias vs human bias 
- 0:10 - 0:40 [Activity 4] Identifying bias in corpora (occupations, gender, ...) [GloVe]
- 0:40 - 0:50 Towards unbiased embeddings; Examine “debiased” embeddings
- 0:50 - 0:60 Concluding remarks and debate

In [ ]:
!pip install numpy nltk scikit-learn matplotlib gensim seaborn plotly;
!python -m nltk.downloader all;

In [ ]:
import numpy as np
import nltk
# import plotly.plotly as py
import sklearn
import matplotlib.pyplot as plt
import gensim

from IPython.display import HTML

<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Your turn: Generate a vector including integers from 4 and 8 of size 10
<br>
<em>
<strong>Hint</strong>: Use the numpy functions
</em>
</p>
</div>

First we will work through examples from a paper that came out in 2016 on debiasing gender vectors. (https://arxiv.org/pdf/1607.06520.pdf)


In [ ]:
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
import numpy.linalg as la

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# We first load our embedddings with the code the authors provide
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')

In [ ]:
# The authors also give us a hand-picked set of definitional gender pairs, pairs of gender words we want to equalize, and words that are gender-specific  

with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

In [ ]:
defs

In [ ]:
equalize_pairs

In [ ]:
gender_specific_words

In [ ]:
# We also get a list of professions
professions = load_professions()

In [ ]:
professions

What is the problem we're dealing with here? Where is the 'bias'?

In [ ]:
# We identify a gender subspace or direction by subtracting the 'he' vector from the 'she' vector
gender_vector = E.diff('she','he')

In [ ]:
# Using cosine similarity, see how gender specific words (such as 'boyfriend' or 'actress') relate to the gender vector
for w in gender_specific_words:
    if w in E.index:
        print (w, cosine_similarity(E.v(w).reshape(1,-1), gender_vector.reshape(1,-1)))

In [ ]:
# Activity: Make your own list of words that you think will be demonstrate or embed bias.


In [ ]:
# We will look at how these 'biases' are reflected in our hand-picked set of profession-related words.
for w in [p[0] for p in professions]:
    if w in E.index:
        print (w, cosine_similarity(E.v(w).reshape(1,-1), gender_vector.reshape(1,-1)))

While as humanists and social scientists we might find such discrepancies, or rather consistent biases, interesting data, computer scientists found this embedded bias deeply problematic (and perhaps horrifying). So, they have tried to correct these discrepancies.

In [ ]:
E.v('physicist')

In [ ]:
physicist = E.v('physicist')/la.norm(E.v('physicist'))

In [ ]:
factor = np.dot(physicist,gender_vector.T)


In [ ]:
bias_comp = np.multiply(gender_vector, factor)


In [ ]:
debiased = physicist - bias_comp


In [ ]:
cosine_similarity(debiased.reshape(1,-1), gender_vector.reshape(1,-1))

In [ ]:
cosine_similarity(physicist.reshape(1,-1), gender_vector.reshape(1,-1))

In [ ]:
before_debiasing = sorted([(w,cosine_similarity(E.v(w).reshape(1,-1),gender_vector.reshape(1,-1))) for w in profession_words], key=lambda x:x[1][[0]])
before_debiasing[:15], before_debiasing[-15:]

In [ ]:
from debiaswe.debias import debias

In [ ]:
debias(E, gender_specific_words, defs, equalize_pairs)

In [ ]:
after_debiasing = sorted([(w,cosine_similarity(E.v(w).reshape(1,-1),gender_vector.reshape(1,-1))) for w in profession_words], key=lambda x:x[1][[0]])

In [ ]:
after_debiasing[:15], after_debiasing[-15:]

In [ ]:
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

In [ ]:
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

In [ ]:
sp[:30]

In [ ]:
with open('./glove/glove.6B.50d.txt','rb') as lines:
    w2v = {line.split()[0].decode('utf-8'): np.array(list(map(float, line.split()[1:]))).reshape(1,50)
           for line in lines}
    


$S_{(a,b)}(x,y) = 
  \left\{
\begin{array}{ll}
      cos(\vec{a} - \vec{b},\vec{x} - \vec{y})& if \left\Vert \vec{x} - \vec{y} \right\Vert \leq \delta \\
      0 & otherwise \\
\end{array} 
\right. $

In [ ]:
a_gender = E.best_analogies_dist_thresh(v_gender)

In [ ]:
print(a_gender[:30])

In [ ]:
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])